In [1]:
import pandas as pd

df = pd.read_csv('../../dataset/tables.csv')
df

,id,caption,table,footnotes,references,id_file
0,S3.T1,Table 1: The cosine similarity between the pri...,"<table class=""ltx_tabular ltx_centering ltx_gu...",[],[],./extraction/2404.12008v3.json
1,S5.T2,Table 2: Performance comparison in terms of ND...,"<table class=""ltx_tabular ltx_centering ltx_gu...",[],['Comparison under three testing paradigms. Ta...,./extraction/2404.12008v3.json
2,S5.T3,Table 3: NDCG@20 comparison with LightGCN and ...,"<table class=""ltx_tabular ltx_centering ltx_gu...",[],"['To investigate the adaptability of ReSN, we ...",./extraction/2404.12008v3.json
3,A2.T4,Table 4: The proportion of interactions of the...,"<table class=""ltx_tabular ltx_centering ltx_al...",[],[],./extraction/2404.12008v3.json
4,A2.T5,"Table 5: The value of <math alttext=""\sigma_{1...","<table class=""ltx_tabular ltx_centering ltx_gu...",[],['Table 5 presents the values of <math alttext...,./extraction/2404.12008v3.json
...,...,...,...,...,...,...
3368,A1.T2,Table 2. Zero-Shot Performance Comparison Acro...,"<table class=""ltx_tabular ltx_centering ltx_gu...",[],['Response Variance.\nTo estimate the variance...,./extraction/2410.13604v1.json
3369,S5.T1,Table 1. Summary of population-level evaluatio...,"<table class=""ltx_tabular ltx_centering ltx_gu...",[],['We summarize results in Table 1. These analy...,./extraction/2410.13680v1.json
3370,S6.T2,"Table 2. Kendall’s <math alttext=""\tau"" class=...","<table class=""ltx_tabular ltx_guessed_headers ...",[],['Results for corpora using average precision ...,./extraction/2410.13680v1.json
3371,S6.T3,"Table 3. Kendall’s <math alttext=""\tau"" class=...","<table class=""ltx_tabular ltx_guessed_headers ...",[],['Results for corpora using average precision ...,./extraction/2410.13680v1.json


# Preprocessing
splitting the data into 2 dataframes:
- tables
- tables_columns

In [2]:
import re
from lxml import html
from nltk.stem import PorterStemmer
from unidecode import unidecode

def get_table_columns(html_input):
    # Clean the HTML input
    html_input = html_input.replace('\n', '')
    tree = html.fromstring(html_input)

    # Extract the first row (header)
    header_row = tree.xpath('//tr[1]')

    if header_row:
        columns = []
        # Find all cells (<th> or <td>) in the first row
        cells = header_row[0].xpath('.//th | .//td')

        for cell in cells:
            # Extract regular text or the alternative text from <math> elements
            cell_text = ''.join(cell.xpath('.//text()')).strip()
            alttext = cell.xpath('.//math/@alttext')

            if alttext:
                cell_text = alttext[0]

            # Add the cell text to columns if it's not empty
            if cell_text:
                columns.append(clean_text(cell_text))
        return columns
    else:
        return []

def get_table_rows(html_input):
    # Analizza l'input HTML
    tree = html.fromstring(html_input)
    
    # Utilizza XPath per estrarre le righe della tabella
    rows = []
    for row in tree.xpath('//tr')[1:]:
        # Estrae il testo da tutte le celle (sia th che td) in una riga
        cells = row.xpath('.//th//text() | .//td//text()')

        # Aggiunge la riga pulita alla lista delle righe
        if cells:
            rows.append(clean_text(cells))
    
    return rows

def clean_text(text):
    # Remove leading and trailing whitespaces
    #cast to string
    text = str(text)

    text = text.strip()
    
    # Remove HTML tags and attributes
    text = re.sub(r'<[^>]*>', '', text)
    
    # De-accent
    text = unidecode(text)
    
    # sub dash underscore  with pipe
    text = re.sub(r'[-_]', '|', text)
    
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    # Stemming
    stemmer = PorterStemmer()
    text = '|'.join([stemmer.stem(word) for word in text.split()])
    
    return text

def get_id_file(path):
    file = path.split('/')[-1]
    return '.'.join(file.split('.')[:2])



In [3]:
# df_tables is the dataframe that contains the tables df

df_tables = df.copy()

#clean the id_file column
df_tables['id_file'] = df_tables['id_file'].apply(get_id_file)

df_tables['columns'] = df_tables['table'].apply(lambda x: get_table_columns(x))
df_tables['n_columns'] = df_tables['columns'].apply(len)

df_tables['rows'] = df_tables['table'].apply(lambda x: get_table_rows(x))
df_tables['n_rows'] = df_tables['rows'].apply(len)

df_tables['n_cells'] = df_tables['n_columns'] * df_tables['n_rows']
df_tables = df_tables[df_tables['n_cells'] > 0]

df_tables['caption'] = df_tables['caption'].apply(clean_text)
df_tables['caption_len'] = df_tables['caption'].apply(lambda x: len(x) if type(x) == str else 0)

df_tables['n_references'] = df_tables['references'].apply(lambda x: len([item for item in x[1:-1].split(', ') if item]) if isinstance(x, str) else 0)
df_tables['references'] = df_tables['references'].apply(lambda x: [clean_text(y) for y in x[1:-1].split(', ')] if isinstance(x, str) else [])

df_tables['n_footnotes'] = df_tables['footnotes'].apply(lambda x: len([item for item in x[1:-1].split(', ') if item]) if isinstance(x, str) else 0)
df_tables['footnotes'] = df_tables['footnotes'].apply(lambda x: [clean_text(y) for y in x[1:-1].split(', ')] if isinstance(x, str) else [])

df_tables = df_tables.drop(columns=['table'])

df_tables

,id,caption,footnotes,references,id_file,columns,n_columns,rows,n_rows,n_cells,caption_len,n_references,n_footnotes
0,S3.T1,tabl|1|the|cosin|similar|between|the|princip|s...,[],[],2404.12008v3,"[backbon, movielen, douban, globo]",4,"[mse|bce|bpr|mse|bce|bpr|mse|bce|bpr, mf|0993|...",4,16,643,0,0
1,S5.T2,tabl|2|perform|comparison|in|term|of|ndcg|betw...,[],[comparison|under|three|test|paradigm|tabl|2|s...,2404.12008v3,"[movielen, douban, yelp2018, gowalla, globo, y...",7,[com|deb|com|deb|com|deb|com|deb|com|deb|uni|u...,8,56,698,8,0
2,S5.T3,tabl|3|ndcg20|comparison|with|lightgcn|and|xsi...,[],"[to|investig|the|adapt|of|resn, we|evalu|it|wi...",2404.12008v3,"[movielen, douban, gowalla]",3,"[lgcn|xsgcl|lgcn|xsgcl|lgcn|xsgcl, backbon|015...",10,30,58,9,0
3,A2.T4,tabl|4|the|proport|of|interact|of|the|top|20|o...,[],[],2404.12008v3,"[movielens1m, douban, globo]",3,[673|863|905],1,3,92,0,0
4,A2.T5,tabl|5|the|valu|of|963|1|2|superscript|subscri...,[],[tabl|5|present|the|valu|of|n|n|n|963n|1n|2n|n...,2404.12008v3,"[model, movielen, douban, globo]",4,[s|1|2|superscript|subscript|1|2|sigma12|ital|...,3,12,1341,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3368,A1.T2,tabl|2|zeroshot|perform|comparison|across|30|r...,[],[respons|variancento|estim|the|varianc|of|llm|...,2410.13604v1,"[model, model|name, precision10, recall10, f11...",6,[categori|f|valu|pr|f|f|valu|pr|f|f|valu|pr|f|...,39,234,1239,6,0
3369,S5.T1,tabl|1|summari|of|populationlevel|evalu|proper...,[],[we|summar|result|in|tabl|1|these|analys|all|c...,2410.13680v1,"[pp, au, dp]",3,"[min, lmin, avg, gavg, s10, auc|4|subscript|au...",8,24,587,3,0
3370,S6.T2,tabl|2|kendal|964|120591|tau|italic964|for|var...,[],[result|for|corpora|use|averag|precis|as|the|u...,2410.13680v1,[pessimist|evalu],1,[nrun|lmin|min|gavg|s10|n|auc|4|subscript|auc|...,28,28,1349,26,0
3371,S6.T3,tabl|3|kendal|964|120591|tau|italic964|for|rob...,[],[result|for|corpora|use|averag|precis|as|the|u...,2410.13680v1,[pessimist|evalu],1,[lmin|min|gavg|s10|n|auc|4|subscript|auc|4|tex...,9,9,374,6,0


In [ ]:
# df_tables_columns is from parsing html of th df['table'] column
# df_tables_columns structure: table_id, column_name, file_id

df_tables_columns = pd.DataFrame(columns=['table_id', 'column_name', 'id_file'])

for index, row in df.iterrows():
    columns = get_table_columns(row['table'])
    for col in columns:
        df_tables_columns = pd.concat([df_tables_columns, pd.DataFrame({
            'table_id': row['id'],
            'column_name': [col],
            'id_file': [get_id_file(row['id_file'])]
        })])
    
df_tables_columns



In [7]:
# find column name with most length
df_tables_columns['column_name_len'] = df_tables_columns['column_name'].apply(lambda x: len(x))
df_tables_columns

,table_id,column_name,id_file,column_name_len
0,S3.T1,backbon,2404.12008v3,7
0,S3.T1,movielen,2404.12008v3,8
0,S3.T1,douban,2404.12008v3,6
0,S3.T1,globo,2404.12008v3,5
0,S5.T2,movielen,2404.12008v3,8
...,...,...,...,...
0,S6.T3,pessimist|evalu,2410.13680v1,15
0,A1.T4,request,2410.13680v1,7
0,A1.T4,run,2410.13680v1,3
0,A1.T4,relrequest,2410.13680v1,10


In [5]:
# Save the dataframes to csv files
df_tables.to_csv('../../dataset/tables_clean.csv', index=False)
df_tables_columns.to_csv('../../dataset/tables_columns_clean.csv', index=False)